In [2]:
%load_ext Cython

In [3]:
%%cython -a

class ExpectiminimaxValuePlayer():

    # Konstruktor braucht einen Parameter für die maximal Suchtiefe
    # 0 = 1-ply, 1= 2-ply, 2 = 3-ply, usw.
    def __init__(self, str player, valuefunction, int max_depth):
        self.player = player
        self.value = valuefunction
        self.max_depth = max_depth
    
    def get_action(self, actions, game):
        cdef float best_value, value
        # Spielstatus speichern
        old_state = game.get_state()
        # Variablen initialisieren
        best_value = -1
        best_action = None
        # Alle Züge durchsuchen
        for a in actions:
            # Zug ausführen
            game.execute_moves(a, self.player)
            # Spielstatus bewerten
            value = self.expectiminimax(game, 0)
            # Besten merken
            if value > best_value:
                best_value = value
                best_action = a
            # Spiel zurücksetzen
            game.reset_to_state(old_state)
        return best_action
        
    def expectiminimax(self, game, int depth):
        cdef float value, v, probability, temp_val
        # Blatt in unserem Baum
        if depth == self.max_depth:
            return self.value(game, self.player)
        else:
            # Alle möglichen Würfe betrachten
            all_rolls = [(a,b) for a in range(1,7) for b in range(a,7)]
            value = 0
            for roll in all_rolls:
                # Wahrscheinlichkeiten von jedem Wurf
                probability = 1/18 if roll[0] != roll[1] else 1/36
                state = game.get_state()
                # Min-Knoten
                if depth % 2 == 0:
                    moves = game.get_moves(roll, game.get_opponent(self.player))
                    temp_val = 1
                    for move in moves:
                        game.execute_moves(move, game.get_opponent(self.player))
                        # Bewertet wird aber aus unserer Perspektive
                        v = self.expectiminimax(game, depth + 1)
                        if v < temp_val:
                            temp_val = v
                # Max-Knoten
                else:
                    moves = game.get_moves(roll, self.player)
                    temp_val = 0
                    for move in moves:
                        game.execute_moves(move, self.player)
                        # Bewertet wird aber aus unserer Perspektive
                        v = self.expectiminimax(game, depth + 1)
                        if v > temp_val:
                            temp_val = v
                # Spiel zurücksetzen    
                game.reset_to_state(state)
                # Wert gewichtet addieren
                value += probability * temp_val
            return value
    
    def get_name(self):
        return "cExpectiminimaxValuePlayer [" + self.value.__name__ + "]"

In [7]:
import time
from CythonBackgammon import Game 

# Testfunktion für Player

def test(players, games=100, debug=False):
    wins = {Game.PLAYERS[0] : 0, Game.PLAYERS[1] : 0}
    # Zeit messen und Spielen, diesmal ohne loggen
    start = time.time()
    for i in range(games):
        game = Game()
        winner = game.play(players, debug=debug)
        wins[winner] += 1
        win_num = 0 if winner == game.players[0] else 1
        print("Spiel", i, "von", games ,"geht an", players[win_num].get_name(), "(" , winner , ")")
    end = time.time()

    # Hübsch ausgeben
    print()
    print(wins)
    print(games, "Spiele in ", end - start, "Sekunden")

Diese Variante vom PlayerTest ersetzt nun die im PlayerTest.py

In [9]:
import Player

players = [ExpectiminimaxValuePlayer('black', Player.singleton, 1), Player.ValuePlayer('white', Player.singleton)]
test(players, 100)

Spiel 0 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 1 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 2 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 3 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 4 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 5 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 6 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 7 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 8 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 10 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 11 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 12 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 13 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 14 von 100 geht an cExpectiminimaxValuePlayer [singleton] 

#### Performance

235 Sekunden mit cython expectiminimax 2-ply, statt 523 Sekunden mit Python 2-ply!

Und das ohne groß C-Funktionen zu nutzen, da ist noch deutlich Luft nach oben, wenn man sich in C auskennt.

Vielleicht ist Three-Ply jetzt erträgllich!

In [10]:
import Player

players = [ExpectiminimaxValuePlayer('black', Player.singleton, 2), Player.ValuePlayer('white', Player.singleton)]
test(players, 100)

Spiel 0 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 1 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 2 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 3 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 4 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 5 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 6 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 7 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )
Spiel 8 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 100 geht an cExpectiminimaxValuePlayer [singleton] ( black )


KeyboardInterrupt: 

__Negativ__

### Nächster Schritt

Mehr Teile die in Schleifen sehr oft ausgeführt werden in C schreiben, eventuell dann sogar parallelisieren!

In [16]:
%%cython -a

import ctypes

cdef struct gameState:
    
    int old_state[24]
    int points[24]
    int black_taken
    int white_taken
    
cdef gameState init_gameState(int[24] points, int black_taken, int white_taken):
    cdef gameState state
    state.old_state = points
    state.points = points
    state.black_taken = black_taken
    state.white_taken = white_taken
    return state
      
# player_value = 1 für Schwarz, -1 für Weiß
cdef gameState execute_move_and_get_new_state(gameState state, int start_pos, int end_pos, int player_value):
    #Stein von der alten Position nehmen, falls nicht auf der Bar
    if start_pos != -1:
        state.points[start_pos] -= player_value
    #Stein auf die gewünschte Stelle setzen, falls noch auf dem Spielfeld
    if end_pos >= 0 and end_pos < 24:
        #Falls dort bereits ein Gegnerstein war wird er auf die Bar gelegt
        if player_value == 1 and state.points[end_pos] == -1:
            state.points[end_pos] = 0
            state.white_taken += 1
        elif player_value == -1 and state.points[end_pos] == 1:
            state.points[end_pos] = 0
            state.black_taken += 1
        #Stein platzieren
        state.points[end_pos] += player_value
    return state

cdef gameState reset(gameState s):
    s.points = s.old_state
    return s

cdef float singleton(gameState state, int player_value):
    cdef int singles, i
    singles = 0
    for point in state.points[:24]:
        if point == player_value:
            singles += 1
    # Zahl in [0,1]
    return (15 - singles) / 15

cdef float way_to_go(gameState state, int player_value):
    cdef int steps, bar_chk, i
    # Schritte berechnen
    steps = 0
    # Steine auf dem Spielfeld
    for i in state.points[:24]:
        if player_value == 1:
            steps += (24 - i) * state.points[i]
        else:
            steps -= i * state.points[i]
    # Steine auf der Bar
    bar_chk = state.white_taken if player_value == 1 else state.black_taken
    steps += 25 * bar_chk 
    return steps / 375

class cSingletonPlayer():

    def __init__(self, str player):
        self.p = player

    def get_action(self, actions, game):
        cdef gameState state
        cdef float value, best_value
        cdef int points_arr[24]
        cdef int player_value, m1, m2
        player_value = 0
        # Spielstatus speichern
        points_arr = (ctypes.c_int * len(game.points))(*game.points)
        state = init_gameState(points_arr, game.black_taken, game.white_taken)
        # Variablen initialisieren
        if self.p == 'black':
            player_value = 1
        else:
            player_value = -1
        best_value = -1.0
        best_action = None
        # Alle Züge durchsuchen
        for action in actions:
            #Jeden Unterzug ausführen
            for (a, m2) in action:
                if a == 'bar':
                    m1 = -1
                else:
                    m1 = a
                # Zug ausführen
                state = execute_move_and_get_new_state(state, m1, m2, player_value)
            # Spielstatus bewerten
            value = singleton(state, player_value)
            # Besten merken
            if value > best_value:
                best_value = value
                best_action = action
            # Spiel zurücksetzen
            state = reset(state)
        return best_action
    
    def get_name(self):
        return "cSingletonPlayer"

In [17]:
import Player
import PlayerTest

players = [cSingletonPlayer('black'), cSingletonPlayer('white')]    
PlayerTest.test(players, games=1, debug=True)

((12, 9), (9, 5))
('Current Player:', 'white')
('Moves:', [((23, 20), (12, 8)), ((23, 19), (12, 9)), ((23, 20), (7, 3)), ((23, 19), (7, 4)), ((23, 20), (5, 1)), ((23, 19), (5, 2)), ((12, 9), (7, 3)), ((12, 8), (7, 4)), ((12, 9), (5, 1)), ((12, 8), (5, 2)), ((7, 4), (5, 1)), ((7, 3), (5, 2)), ((23, 20), (23, 19)), ((12, 9), (12, 8)), ((12, 9), (9, 5)), ((7, 4), (7, 3)), ((5, 2), (5, 1))])
('Roll:', (3, 4), '| Move:', ((12, 9), (9, 5)))
('GameState:', [2, 0, 0, 0, 0, -6, 0, -3, 0, 0, 0, 5, -4, 0, 0, 0, 3, 0, 5, 0, 0, 0, 0, -2], '|', [0, 11, 16, 18], '|', [23, 12, 7, 5])
('Black/White:', 15, '/', -15, 'Bar:', 0, '/', 0)
()
((16, 21), (18, 21))
('Current Player:', 'black')
('Moves:', [((0, 3), (11, 16)), ((0, 3), (16, 21)), ((11, 14), (16, 21)), ((11, 16), (16, 19)), ((11, 16), (18, 21)), ((16, 21), (18, 21)), ((0, 3), (3, 8)), ((11, 14), (11, 16)), ((11, 14), (14, 19)), ((16, 19), (16, 21))])
('Roll:', (3, 5), '| Move:', ((16, 21), (18, 21)))
('GameState:', [2, 0, 0, 0, 0, -6, 0, -3, 0, 0

In [19]:
PlayerTest.test(players, games=1000)

Spiel 0 von 1000 geht an cSingletonPlayer ( white )
Spiel 1 von 1000 geht an cSingletonPlayer ( white )
Spiel 2 von 1000 geht an cSingletonPlayer ( black )
Spiel 3 von 1000 geht an cSingletonPlayer ( black )
Spiel 4 von 1000 geht an cSingletonPlayer ( white )
Spiel 5 von 1000 geht an cSingletonPlayer ( black )
Spiel 6 von 1000 geht an cSingletonPlayer ( black )
Spiel 7 von 1000 geht an cSingletonPlayer ( white )
Spiel 8 von 1000 geht an cSingletonPlayer ( white )
Spiel 9 von 1000 geht an cSingletonPlayer ( black )
Spiel 10 von 1000 geht an cSingletonPlayer ( white )
Spiel 11 von 1000 geht an cSingletonPlayer ( white )
Spiel 12 von 1000 geht an cSingletonPlayer ( white )
Spiel 13 von 1000 geht an cSingletonPlayer ( white )
Spiel 14 von 1000 geht an cSingletonPlayer ( black )
Spiel 15 von 1000 geht an cSingletonPlayer ( white )
Spiel 16 von 1000 geht an cSingletonPlayer ( white )
Spiel 17 von 1000 geht an cSingletonPlayer ( white )
Spiel 18 von 1000 geht an cSingletonPlayer ( white )
Spi

Spiel 182 von 1000 geht an cSingletonPlayer ( white )
Spiel 183 von 1000 geht an cSingletonPlayer ( white )
Spiel 184 von 1000 geht an cSingletonPlayer ( white )
Spiel 185 von 1000 geht an cSingletonPlayer ( black )
Spiel 186 von 1000 geht an cSingletonPlayer ( black )
Spiel 187 von 1000 geht an cSingletonPlayer ( black )
Spiel 188 von 1000 geht an cSingletonPlayer ( white )
Spiel 189 von 1000 geht an cSingletonPlayer ( black )
Spiel 190 von 1000 geht an cSingletonPlayer ( black )
Spiel 191 von 1000 geht an cSingletonPlayer ( white )
Spiel 192 von 1000 geht an cSingletonPlayer ( black )
Spiel 193 von 1000 geht an cSingletonPlayer ( white )
Spiel 194 von 1000 geht an cSingletonPlayer ( black )
Spiel 195 von 1000 geht an cSingletonPlayer ( white )
Spiel 196 von 1000 geht an cSingletonPlayer ( black )
Spiel 197 von 1000 geht an cSingletonPlayer ( white )
Spiel 198 von 1000 geht an cSingletonPlayer ( black )
Spiel 199 von 1000 geht an cSingletonPlayer ( black )
Spiel 200 von 1000 geht an c

Spiel 348 von 1000 geht an cSingletonPlayer ( white )
Spiel 349 von 1000 geht an cSingletonPlayer ( white )
Spiel 350 von 1000 geht an cSingletonPlayer ( white )
Spiel 351 von 1000 geht an cSingletonPlayer ( black )
Spiel 352 von 1000 geht an cSingletonPlayer ( white )
Spiel 353 von 1000 geht an cSingletonPlayer ( white )
Spiel 354 von 1000 geht an cSingletonPlayer ( white )
Spiel 355 von 1000 geht an cSingletonPlayer ( black )
Spiel 356 von 1000 geht an cSingletonPlayer ( white )
Spiel 357 von 1000 geht an cSingletonPlayer ( white )
Spiel 358 von 1000 geht an cSingletonPlayer ( white )
Spiel 359 von 1000 geht an cSingletonPlayer ( white )
Spiel 360 von 1000 geht an cSingletonPlayer ( white )
Spiel 361 von 1000 geht an cSingletonPlayer ( black )
Spiel 362 von 1000 geht an cSingletonPlayer ( white )
Spiel 363 von 1000 geht an cSingletonPlayer ( white )
Spiel 364 von 1000 geht an cSingletonPlayer ( white )
Spiel 365 von 1000 geht an cSingletonPlayer ( black )
Spiel 366 von 1000 geht an c

Spiel 515 von 1000 geht an cSingletonPlayer ( white )
Spiel 516 von 1000 geht an cSingletonPlayer ( black )
Spiel 517 von 1000 geht an cSingletonPlayer ( white )
Spiel 518 von 1000 geht an cSingletonPlayer ( black )
Spiel 519 von 1000 geht an cSingletonPlayer ( white )
Spiel 520 von 1000 geht an cSingletonPlayer ( white )
Spiel 521 von 1000 geht an cSingletonPlayer ( white )
Spiel 522 von 1000 geht an cSingletonPlayer ( white )
Spiel 523 von 1000 geht an cSingletonPlayer ( white )
Spiel 524 von 1000 geht an cSingletonPlayer ( black )
Spiel 525 von 1000 geht an cSingletonPlayer ( white )
Spiel 526 von 1000 geht an cSingletonPlayer ( white )
Spiel 527 von 1000 geht an cSingletonPlayer ( black )
Spiel 528 von 1000 geht an cSingletonPlayer ( black )
Spiel 529 von 1000 geht an cSingletonPlayer ( white )
Spiel 530 von 1000 geht an cSingletonPlayer ( white )
Spiel 531 von 1000 geht an cSingletonPlayer ( black )
Spiel 532 von 1000 geht an cSingletonPlayer ( white )
Spiel 533 von 1000 geht an c

Spiel 676 von 1000 geht an cSingletonPlayer ( white )
Spiel 677 von 1000 geht an cSingletonPlayer ( black )
Spiel 678 von 1000 geht an cSingletonPlayer ( white )
Spiel 679 von 1000 geht an cSingletonPlayer ( white )
Spiel 680 von 1000 geht an cSingletonPlayer ( white )
Spiel 681 von 1000 geht an cSingletonPlayer ( white )
Spiel 682 von 1000 geht an cSingletonPlayer ( white )
Spiel 683 von 1000 geht an cSingletonPlayer ( black )
Spiel 684 von 1000 geht an cSingletonPlayer ( black )
Spiel 685 von 1000 geht an cSingletonPlayer ( black )
Spiel 686 von 1000 geht an cSingletonPlayer ( white )
Spiel 687 von 1000 geht an cSingletonPlayer ( black )
Spiel 688 von 1000 geht an cSingletonPlayer ( white )
Spiel 689 von 1000 geht an cSingletonPlayer ( white )
Spiel 690 von 1000 geht an cSingletonPlayer ( white )
Spiel 691 von 1000 geht an cSingletonPlayer ( white )
Spiel 692 von 1000 geht an cSingletonPlayer ( white )
Spiel 693 von 1000 geht an cSingletonPlayer ( white )
Spiel 694 von 1000 geht an c

Spiel 848 von 1000 geht an cSingletonPlayer ( white )
Spiel 849 von 1000 geht an cSingletonPlayer ( white )
Spiel 850 von 1000 geht an cSingletonPlayer ( black )
Spiel 851 von 1000 geht an cSingletonPlayer ( black )
Spiel 852 von 1000 geht an cSingletonPlayer ( white )
Spiel 853 von 1000 geht an cSingletonPlayer ( white )
Spiel 854 von 1000 geht an cSingletonPlayer ( white )
Spiel 855 von 1000 geht an cSingletonPlayer ( black )
Spiel 856 von 1000 geht an cSingletonPlayer ( white )
Spiel 857 von 1000 geht an cSingletonPlayer ( white )
Spiel 858 von 1000 geht an cSingletonPlayer ( white )
Spiel 859 von 1000 geht an cSingletonPlayer ( black )
Spiel 860 von 1000 geht an cSingletonPlayer ( black )
Spiel 861 von 1000 geht an cSingletonPlayer ( white )
Spiel 862 von 1000 geht an cSingletonPlayer ( white )
Spiel 863 von 1000 geht an cSingletonPlayer ( white )
Spiel 864 von 1000 geht an cSingletonPlayer ( white )
Spiel 865 von 1000 geht an cSingletonPlayer ( black )
Spiel 866 von 1000 geht an c

In [20]:
players = [cSingletonPlayer('black'), Player.ValuePlayer('white', Player.singleton)]  
PlayerTest.test(players, games=1000)

Spiel 0 von 1000 geht an cSingletonPlayer ( black )
Spiel 1 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 2 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 3 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 4 von 1000 geht an cSingletonPlayer ( black )
Spiel 5 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 6 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 7 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 8 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 10 von 1000 geht an cSingletonPlayer ( black )
Spiel 11 von 1000 geht an cSingletonPlayer ( black )
Spiel 12 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 13 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 14 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 15 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 16 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 17 von 1000 

Spiel 144 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 145 von 1000 geht an cSingletonPlayer ( black )
Spiel 146 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 147 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 148 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 149 von 1000 geht an cSingletonPlayer ( black )
Spiel 150 von 1000 geht an cSingletonPlayer ( black )
Spiel 151 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 152 von 1000 geht an cSingletonPlayer ( black )
Spiel 153 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 154 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 155 von 1000 geht an cSingletonPlayer ( black )
Spiel 156 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 157 von 1000 geht an cSingletonPlayer ( black )
Spiel 158 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 159 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 160 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel

Spiel 290 von 1000 geht an cSingletonPlayer ( black )
Spiel 291 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 292 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 293 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 294 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 295 von 1000 geht an cSingletonPlayer ( black )
Spiel 296 von 1000 geht an cSingletonPlayer ( black )
Spiel 297 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 298 von 1000 geht an cSingletonPlayer ( black )
Spiel 299 von 1000 geht an cSingletonPlayer ( black )
Spiel 300 von 1000 geht an cSingletonPlayer ( black )
Spiel 301 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 302 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 303 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 304 von 1000 geht an cSingletonPlayer ( black )
Spiel 305 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 306 von 1000 geht an cSingletonPlayer ( black )
Spiel 307 von 1000 

Spiel 433 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 434 von 1000 geht an cSingletonPlayer ( black )
Spiel 435 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 436 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 437 von 1000 geht an cSingletonPlayer ( black )
Spiel 438 von 1000 geht an cSingletonPlayer ( black )
Spiel 439 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 440 von 1000 geht an cSingletonPlayer ( black )
Spiel 441 von 1000 geht an cSingletonPlayer ( black )
Spiel 442 von 1000 geht an cSingletonPlayer ( black )
Spiel 443 von 1000 geht an cSingletonPlayer ( black )
Spiel 444 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 445 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 446 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 447 von 1000 geht an cSingletonPlayer ( black )
Spiel 448 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 449 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 450 von 1000 

Spiel 609 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 610 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 611 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 612 von 1000 geht an cSingletonPlayer ( black )
Spiel 613 von 1000 geht an cSingletonPlayer ( black )
Spiel 614 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 615 von 1000 geht an cSingletonPlayer ( black )
Spiel 616 von 1000 geht an cSingletonPlayer ( black )
Spiel 617 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 618 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 619 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 620 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 621 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 622 von 1000 geht an cSingletonPlayer ( black )
Spiel 623 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 624 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 625 von 1000 geht an cSingletonPlayer ( black )
Spiel

Spiel 778 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 779 von 1000 geht an cSingletonPlayer ( black )
Spiel 780 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 781 von 1000 geht an cSingletonPlayer ( black )
Spiel 782 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 783 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 784 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 785 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 786 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 787 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 788 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 789 von 1000 geht an cSingletonPlayer ( black )
Spiel 790 von 1000 geht an cSingletonPlayer ( black )
Spiel 791 von 1000 geht an cSingletonPlayer ( black )
Spiel 792 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 793 von 1000 geht an cSingletonPlayer ( black )
Spiel 794 von 1000 geht an cSingletonPlayer ( black )
Spiel 795 vo

Spiel 940 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 941 von 1000 geht an cSingletonPlayer ( black )
Spiel 942 von 1000 geht an cSingletonPlayer ( black )
Spiel 943 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 944 von 1000 geht an cSingletonPlayer ( black )
Spiel 945 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 946 von 1000 geht an cSingletonPlayer ( black )
Spiel 947 von 1000 geht an cSingletonPlayer ( black )
Spiel 948 von 1000 geht an cSingletonPlayer ( black )
Spiel 949 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 950 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 951 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 952 von 1000 geht an cSingletonPlayer ( black )
Spiel 953 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 954 von 1000 geht an cSingletonPlayer ( black )
Spiel 955 von 1000 geht an cSingletonPlayer ( black )
Spiel 956 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 957 von 1000 geht an

In [21]:
players = [Player.ValuePlayer('black', Player.singleton), cSingletonPlayer('white')]
PlayerTest.test(players, games=1000)

Spiel 0 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 1 von 1000 geht an cSingletonPlayer ( white )
Spiel 2 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 3 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 4 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 5 von 1000 geht an cSingletonPlayer ( white )
Spiel 6 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 7 von 1000 geht an cSingletonPlayer ( white )
Spiel 8 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 9 von 1000 geht an cSingletonPlayer ( white )
Spiel 10 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 11 von 1000 geht an cSingletonPlayer ( white )
Spiel 12 von 1000 geht an cSingletonPlayer ( white )
Spiel 13 von 1000 geht an cSingletonPlayer ( white )
Spiel 14 von 1000 geht an cSingletonPlayer ( white )
Spiel 15 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 16 von 1000 geht an cSingletonPlayer ( white )
Spiel 17 von 1000 geht an ValuePlayer [singleton] ( b

Spiel 175 von 1000 geht an cSingletonPlayer ( white )
Spiel 176 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 177 von 1000 geht an cSingletonPlayer ( white )
Spiel 178 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 179 von 1000 geht an cSingletonPlayer ( white )
Spiel 180 von 1000 geht an cSingletonPlayer ( white )
Spiel 181 von 1000 geht an cSingletonPlayer ( white )
Spiel 182 von 1000 geht an cSingletonPlayer ( white )
Spiel 183 von 1000 geht an cSingletonPlayer ( white )
Spiel 184 von 1000 geht an cSingletonPlayer ( white )
Spiel 185 von 1000 geht an cSingletonPlayer ( white )
Spiel 186 von 1000 geht an cSingletonPlayer ( white )
Spiel 187 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 188 von 1000 geht an cSingletonPlayer ( white )
Spiel 189 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 190 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 191 von 1000 geht an cSingletonPlayer ( white )
Spiel 192 von 1000 geht an cSingletonPlayer ( w

Spiel 329 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 330 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 331 von 1000 geht an cSingletonPlayer ( white )
Spiel 332 von 1000 geht an cSingletonPlayer ( white )
Spiel 333 von 1000 geht an cSingletonPlayer ( white )
Spiel 334 von 1000 geht an cSingletonPlayer ( white )
Spiel 335 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 336 von 1000 geht an cSingletonPlayer ( white )
Spiel 337 von 1000 geht an cSingletonPlayer ( white )
Spiel 338 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 339 von 1000 geht an cSingletonPlayer ( white )
Spiel 340 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 341 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 342 von 1000 geht an cSingletonPlayer ( white )
Spiel 343 von 1000 geht an cSingletonPlayer ( white )
Spiel 344 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 345 von 1000 geht an cSingletonPlayer ( white )
Spiel 346 von 1000 geht an ValueP

Spiel 497 von 1000 geht an cSingletonPlayer ( white )
Spiel 498 von 1000 geht an cSingletonPlayer ( white )
Spiel 499 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 500 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 501 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 502 von 1000 geht an cSingletonPlayer ( white )
Spiel 503 von 1000 geht an cSingletonPlayer ( white )
Spiel 504 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 505 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 506 von 1000 geht an cSingletonPlayer ( white )
Spiel 507 von 1000 geht an cSingletonPlayer ( white )
Spiel 508 von 1000 geht an cSingletonPlayer ( white )
Spiel 509 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 510 von 1000 geht an cSingletonPlayer ( white )
Spiel 511 von 1000 geht an cSingletonPlayer ( white )
Spiel 512 von 1000 geht an cSingletonPlayer ( white )
Spiel 513 von 1000 geht an cSingletonPlayer ( white )
Spiel 514 von 1000 geht an cSingletonPla

Spiel 673 von 1000 geht an cSingletonPlayer ( white )
Spiel 674 von 1000 geht an cSingletonPlayer ( white )
Spiel 675 von 1000 geht an cSingletonPlayer ( white )
Spiel 676 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 677 von 1000 geht an cSingletonPlayer ( white )
Spiel 678 von 1000 geht an cSingletonPlayer ( white )
Spiel 679 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 680 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 681 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 682 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 683 von 1000 geht an cSingletonPlayer ( white )
Spiel 684 von 1000 geht an cSingletonPlayer ( white )
Spiel 685 von 1000 geht an cSingletonPlayer ( white )
Spiel 686 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 687 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 688 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 689 von 1000 geht an cSingletonPlayer ( white )
Spiel 690 von 1000 geht an

Spiel 834 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 835 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 836 von 1000 geht an cSingletonPlayer ( white )
Spiel 837 von 1000 geht an cSingletonPlayer ( white )
Spiel 838 von 1000 geht an cSingletonPlayer ( white )
Spiel 839 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 840 von 1000 geht an cSingletonPlayer ( white )
Spiel 841 von 1000 geht an cSingletonPlayer ( white )
Spiel 842 von 1000 geht an cSingletonPlayer ( white )
Spiel 843 von 1000 geht an cSingletonPlayer ( white )
Spiel 844 von 1000 geht an cSingletonPlayer ( white )
Spiel 845 von 1000 geht an cSingletonPlayer ( white )
Spiel 846 von 1000 geht an cSingletonPlayer ( white )
Spiel 847 von 1000 geht an cSingletonPlayer ( white )
Spiel 848 von 1000 geht an cSingletonPlayer ( white )
Spiel 849 von 1000 geht an cSingletonPlayer ( white )
Spiel 850 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 851 von 1000 geht an cSingletonPlayer ( white )


In [74]:
players = [Player.ValuePlayer('black', Player.singleton), Player.ValuePlayer('white', Player.singleton)]
PlayerTest.test(players, games=1000)

Spiel 0 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 1 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 2 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 3 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 4 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 5 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 6 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 7 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 8 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 10 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 11 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 12 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 13 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 14 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 15 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 16 von 1000 geht an ValuePlayer [singleton] 

Spiel 149 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 150 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 151 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 152 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 153 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 154 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 155 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 156 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 157 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 158 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 159 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 160 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 161 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 162 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 163 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 164 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 165 von 1000 geht 

Spiel 287 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 288 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 289 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 290 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 291 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 292 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 293 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 294 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 295 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 296 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 297 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 298 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 299 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 300 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 301 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 302 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 303 von 1000 geht 

Spiel 438 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 439 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 440 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 441 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 442 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 443 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 444 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 445 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 446 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 447 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 448 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 449 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 450 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 451 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 452 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 453 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 454 von 1000 geht 

Spiel 613 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 614 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 615 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 616 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 617 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 618 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 619 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 620 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 621 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 622 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 623 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 624 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 625 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 626 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 627 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 628 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 629 von 1000 geht 

Spiel 762 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 763 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 764 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 765 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 766 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 767 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 768 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 769 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 770 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 771 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 772 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 773 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 774 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 775 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 776 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 777 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 778 von 1000 geht 

Spiel 934 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 935 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 936 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 937 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 938 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 939 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 940 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 941 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 942 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 943 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 944 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 945 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 946 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 947 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 948 von 1000 geht an ValuePlayer [singleton] ( black )
Spiel 949 von 1000 geht an ValuePlayer [singleton] ( white )
Spiel 950 von 1000 geht 